In [7]:
import glob
import json

unique = dict()
total_count = 0
total = {}

for res_file in glob.glob("new/*"):
    with open(res_file, 'r') as r_file:
        cur_res = json.load(r_file)
        total_count += len(cur_res.keys())
        repo = res_file[:res_file.find('_')]
        for func_name in cur_res:
            total.setdefault(func_name, 0)
            total[func_name] += 1

            if cur_res[func_name].get('text', None) is not None:
                unique.setdefault(func_name, set())
                unique[func_name] = cur_res[func_name]['text']

In [8]:
len(unique)

443

In [18]:
with open('out.json', 'w') as w_file:
    json.dump(unique, w_file)